In [9]:
import time
import requests
from shapely.geometry import LineString, MultiLineString, Point
from shapely.ops import unary_union

OVERPASS_URL = "https://overpass.kumi.systems/api/interpreter"

def _fetch_street_multiline(street_name: str):
    query = f"""
    [out:json][timeout:60];
    rel["boundary"="administrative"]["admin_level"="4"]["name"="Санкт-Петербург"]->.spb_rel;
    .spb_rel map_to_area -> .spb;

    way(area.spb)["highway"]["name"="{street_name}"];
    out geom;
    """
    r = requests.get(OVERPASS_URL, params={"data": query}, timeout=90)
    r.raise_for_status()
    data = r.json()

    lines = []
    for el in data.get("elements", []):
        if el.get("type") != "way":
            continue
        geom = el.get("geometry") or []
        if len(geom) < 2:
            continue
        coords = [(p["lon"], p["lat"]) for p in geom]
        lines.append(LineString(coords))

    if not lines:
        return None

    return unary_union(lines)

def _extract_points(geom):
    if geom is None or geom.is_empty:
        return []
    gtype = geom.geom_type
    if gtype == "Point":
        return [geom]
    if gtype == "MultiPoint":
        return list(geom.geoms)
    if gtype in ("LineString", "MultiLineString"):
        return []
    if gtype == "GeometryCollection":
        pts = []
        for g in geom.geoms:
            pts.extend(_extract_points(g))
        return pts
    return []

def get_intersection_coordinates(street1: str, street2: str):
    g1 = _fetch_street_multiline(street1)
    g2 = _fetch_street_multiline(street2)
    if g1 is None or g2 is None:
        return set()
    inter = g1.intersection(g2)
    pts = _extract_points(inter)
    res = set()
    for p in pts:
        lat = round(p.y, 6)
        lon = round(p.x, 6)
        res.add((lat, lon))
    return res

In [10]:
test_cases = [
        ("Шуваловский проспект", "Богатырский проспект"),
        ("Невский проспект", "Приморский проспект"),
        ("Невский проспект", "Литейный проспект"),
        ("улица Савушкина", "Приморский проспект"),
        ("Московский проспект", "Бассейная улица"),
        ("улица Дыбенко", "улица Садовая"),
    ]

print(f"{'Улица 1'} | {'Улица 2'} | {'Результат'}\n")
for s1, s2 in test_cases:
    try:
        pts = get_intersection_coordinates(s1, s2)
        if pts:
            res_str = ", ".join([f"({lat:.6f}, {lon:.6f})" for lat, lon in sorted(pts)])
        else:
            res_str = "Пусто (нет пересечения)"
        print(f"{s1} | {s2} | {res_str}")
    except Exception as e:
        print(f"{s1} | {s2} | Ошибка: {e}")
    time.sleep(5)


Улица 1 | Улица 2 | Результат

Шуваловский проспект | Богатырский проспект | (60.007180, 30.198727), (60.008254, 30.200143), (60.008339, 30.200273), (60.008373, 30.200065)
Невский проспект | Приморский проспект | Пусто (нет пересечения)
Невский проспект | Литейный проспект | (59.932616, 30.347986), (59.932731, 30.347992), (59.932756, 30.347781)
улица Савушкина | Приморский проспект | (59.989761, 30.189501)
Московский проспект | Бассейная улица | (59.864822, 30.320459), (59.864828, 30.320913)
улица Дыбенко | улица Садовая | Пусто (нет пересечения)
